In [6]:
! curl "http://localhost:8000"

Not found
This is a 4store SPARQL server 
Check /status/ for more info


In [7]:
import vcf

# REPLACE pjotr.me with your name!!

In [8]:
print("@prefix : <http://pjotr.me/rdf/ns#> .\n")

vcf_reader = vcf.Reader(open('/home/jupyter/jupyter/contrib/gatk_exome.vcf', 'r'))
count=0
for record in vcf_reader:
    name = "chr"+record.CHROM+"_"+str(record.POS)+"_"+str(record.ALT[0])
    print(":"+name)
    print("        :query_id \""+name+"\" ;")
    print("        :chr \""+record.CHROM+"\" ;")
    print("        :pos "+str(record.POS)+" ;")
    print("        :alt \""+str(record.ALT[0])+"\" .")
    count +=1
    if count > 5:
        break


@prefix : <http://pjotr.me/rdf/ns#> .

:chr1_22848972_C
        :query_id "chr1_22848972_C" ;
        :chr "1" ;
        :pos 22848972 ;
        :alt "C" .
:chr1_33133968_C
        :query_id "chr1_33133968_C" ;
        :chr "1" ;
        :pos 33133968 ;
        :alt "C" .
:chr1_60466814_C
        :query_id "chr1_60466814_C" ;
        :chr "1" ;
        :pos 60466814 ;
        :alt "C" .
:chr1_60466840_T
        :query_id "chr1_60466840_T" ;
        :chr "1" ;
        :pos 60466840 ;
        :alt "T" .
:chr1_62516683_C
        :query_id "chr1_62516683_C" ;
        :chr "1" ;
        :pos 62516683 ;
        :alt "C" .
:chr1_66058513_G
        :query_id "chr1_66058513_G" ;
        :chr "1" ;
        :pos 66058513 ;
        :alt "G" .


### Now write the same to file with samples and a few extra triples

In [9]:

import vcf

buf = """
@prefix    :   <http://pjotr.me/rdf/ns#> .
@prefix rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .

"""

vcf_reader = vcf.Reader(open('/home/jupyter/jupyter/contrib/gatk_exome.vcf', 'r'))
for record in vcf_reader:
    name = "chr"+record.CHROM+"_"+str(record.POS)+"_"+str(record.ALT[0])
    buf += ":"+name+"\n"
    buf += "  :variant \""+name+"\" ;\n"
    buf += "  rdf:type :snp ;\n"
    buf += "  :chr \""+record.CHROM+"\" ;\n"
    buf += "  :pos \""+str(record.POS)+"\" ;\n"
    buf += "  :alt \""+str(record.ALT[0])+"\" .\n"
    for sample in record.samples:
        buf += ":"+name+"_"+str(sample.sample)+"\n"
        if sample["DP"]:
            buf += "  :dp   "+str(sample["DP"])+" ;\n"
        buf += "  :vcf :"+name+" .\n"

# Finally we connect a few SNPs

buf += """
:chr1_66058513_G  :controls  :chr1_114354942_G .
:chr1_114354942_G :controls  :chr1_144873962_C .
:chr1_144873962_C :controls  :chr1_66058513_G .
:chr1_144873962_C :controls  :chrX_134713855_A .
"""

# Write to file

print(buf)
f = open("test.rdf","w") # note this file is shared!
f.write(buf)




@prefix    :   <http://pjotr.me/rdf/ns#> .
@prefix rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .

:chr1_22848972_C
  :variant "chr1_22848972_C" ;
  rdf:type :snp ;
  :chr "1" ;
  :pos "22848972" ;
  :alt "C" .
:chr1_22848972_C_CONTROLP25
  :dp   2 ;
  :vcf :chr1_22848972_C .
:chr1_22848972_C_CONTROLP26
  :vcf :chr1_22848972_C .
:chr1_33133968_C
  :variant "chr1_33133968_C" ;
  rdf:type :snp ;
  :chr "1" ;
  :pos "33133968" ;
  :alt "C" .
:chr1_33133968_C_CONTROLP25
  :vcf :chr1_33133968_C .
:chr1_33133968_C_CONTROLP26
  :dp   2 ;
  :vcf :chr1_33133968_C .
:chr1_60466814_C
  :variant "chr1_60466814_C" ;
  rdf:type :snp ;
  :chr "1" ;
  :pos "60466814" ;
  :alt "C" .
:chr1_60466814_C_CONTROLP25
  :vcf :chr1_60466814_C .
:chr1_60466814_C_CONTROLP26
  :dp   2 ;
  :vcf :chr1_60466814_C .
:chr1_60466840_T
  :variant "chr1_60466840_T" ;
  rdf:type :snp ;
  :chr "1" ;
  :pos "60466840" ;
  :alt "T" .
:chr1_60466840_T_CONTROLP25
  :vcf :chr1_60466840_T .
:chr1_60466840_T_CONTROLP26
  

42101

### Validate the file - we can do this in python

In [10]:
# pip3 install --user rdflib
# python3 rdfvalidate.py

import rdflib
g = rdflib.Graph()

result = g.parse('test.rdf', format='n3')
print(len(g))
for stmt in g:
    print(stmt)

# You should see something here...

1418
(rdflib.term.URIRef('http://pjotr.me/rdf/ns#chr6_121632008_A_CONTROLP25'), rdflib.term.URIRef('http://pjotr.me/rdf/ns#vcf'), rdflib.term.URIRef('http://pjotr.me/rdf/ns#chr6_121632008_A'))
(rdflib.term.URIRef('http://pjotr.me/rdf/ns#chr17_3632836_A'), rdflib.term.URIRef('http://pjotr.me/rdf/ns#alt'), rdflib.term.Literal('A'))
(rdflib.term.URIRef('http://pjotr.me/rdf/ns#chr20_29628431_T'), rdflib.term.URIRef('http://pjotr.me/rdf/ns#variant'), rdflib.term.Literal('chr20_29628431_T'))
(rdflib.term.URIRef('http://pjotr.me/rdf/ns#chr11_63883985_C'), rdflib.term.URIRef('http://pjotr.me/rdf/ns#alt'), rdflib.term.Literal('C'))
(rdflib.term.URIRef('http://pjotr.me/rdf/ns#chr1_228465370_G'), rdflib.term.URIRef('http://pjotr.me/rdf/ns#variant'), rdflib.term.Literal('chr1_228465370_G'))
(rdflib.term.URIRef('http://pjotr.me/rdf/ns#chr19_53344051_C'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://pjotr.me/rdf/ns#snp'))
(rdflib.term.URIRef('http

### Here we upload the data to the database

In [11]:
# Equivalent of curl -T $rdf -H 'Content-Type: application/x-turtle' $uri
import requests

myname="pjotr" # change to your username
rdf="test.rdf" # shared file?!
uri="http://localhost:8000/data/http://biobeat.org/"+myname+"/"+rdf
headers = {'content-type': 'application/x-turtle'}

f = open(rdf)
buf = f.read()
r = requests.put(uri, data=buf, headers=headers)
print(uri)
print(r) # should give a 201


http://localhost:8000/data/http://biobeat.org/pjotr/test.rdf
<Response [201]>


### Let's check what is in 4store


In [12]:
!lynx --dump localhost:8000/status/size/

                       SPARQL httpd server status - size

KB pythoncourse

   Segment # quads (s) quads (sr) models resources
   0         2075      +0         3      1230
   1         2179      +0         3      1292
     Total   4254      +0         3      2522


### And do a first SPARQL query


In [13]:

end_point = "http://localhost:8000/sparql/"

query = """
PREFIX : <http://pjotr.me/rdf/pjotr/ns#>
PREFIX rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?o WHERE {
    ?s rdf:type ?o .
} LIMIT 10
"""

r = requests.post(end_point, data={ "query": query, "output": "text" })
# print r.url

print(r.text)


?o
<http://mudibo.me/rdf/ns#snp>
<http://pjotr.me/rdf/ns#snp>
<http://mudibo.me/rdf/ns#snp>
<http://peter.me/rdf/ns#snp>
<http://peter.me/rdf/ns#snp>
<http://mudibo.me/rdf/ns#snp>
<http://mudibo.me/rdf/ns#snp>
<http://peter.me/rdf/ns#snp>
<http://peter.me/rdf/ns#snp>
<http://mudibo.me/rdf/ns#snp>

